In [ ]:
import sys
import os

# Add the parent directory of 'tree.py' to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))


In [ ]:
from stein import *
from tree import *
from mcmc import kingman_mcmc
from recorder import Recorder

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


sequences = [
    "01001010100101010011",
    "01001010100101010011",
    "01001010100101010011"
]

# Convert sequences to numerical format and validate
num_sequences = []
for seq in sequences:
    num_seq = []
    for base in seq:
        if base not in '01':
            raise ValueError(f"Invalid state '{base}' found in sequence. [only '0' and '1'].")
        num_seq.append(int(base))
    num_sequences.append(num_seq)

# Define the number of leaves for each sequence
num_leaves = [13, 21, 16]

# Create the full sequence array
full_sequences = []
for seq, count in zip(num_sequences, num_leaves):
    full_sequences.extend([seq] * count)

print("Full Sequences:")
for seq in full_sequences:
    print(seq)

In [ ]:
import math
import numpy as np
import random
import tskit
import logging
from scipy.stats import gamma

# Configure logging to see debug messages (optional)
logging.basicConfig(level=logging.INFO)

# Import your classes and functions
from tree import Tree  # Ensure this is correctly implemented
from recorder import Recorder  # Updated Recorder class
from kernel import vfk0_imq_trees, make_imq_trees
from stein import kmat, ksd, create_integrand
from thinning import thin_trees
from mcmc import kingman_mcmc  # Ensure the updated kingman_mcmc is correctly implemented

# Parameters
sample_size = 50
n_states = 2
seq_length = 100.0  # Set to the number of steps or adjust accordingly
steps = 99
initial_mutation_rate = 1.0  # Example mutation rate
pi = [0.5, 0.5]  # Example base frequency distribution

# Initialize tree with sequences
tree = Tree(sample_size, n_states=n_states, sequences=full_sequences)
tree.mutation_rate = initial_mutation_rate  # Set initial mutation rate

# Initialize recorder with sample_size and seq_length
recorder = Recorder(sample_size=sample_size, seq_length=steps)

# Run MCMC
acceptance_prob = kingman_mcmc(tree, recorder, pi, steps=steps)

# Extract tree sequence
ts = recorder.tree_sequence()

# Print tree sequence and acceptance probability
print(ts.draw_text())
print("Acceptance probability =", acceptance_prob)

# Write additional information to txt file
recorder.write_additional_info('additional_info.txt')

# Read and print the new txt file
with open('additional_info.txt', 'r') as f:
    print(f.read())

# Access stored mutation rates, log-likelihoods, and gradients from the recorder
mutation_rates = recorder.mutation_rates
log_likelihoods = recorder.log_likelihoods
gradients = recorder.gradients

# Print the additional information
for i, (mutation_rate, log_likelihood, gradient) in enumerate(zip(mutation_rates, log_likelihoods, gradients)):
    print(f"Tree {i}:")
    print(f"Mutation Rate: {mutation_rate}")
    print(f"Log Likelihood: {log_likelihood}")
    print(f"Gradient: {gradient}")
    print()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.plotting import autocorrelation_plot

# Step 1: Organize Data into DataFrame
data = {
    'Step': range(1, len(recorder.log_likelihoods) + 1),
    'Mutation Rate': recorder.mutation_rates,
    'Log Likelihood': recorder.log_likelihoods
}

df = pd.DataFrame(data)

# Step 2: Trace Plots
fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Mutation Rate Trace Plot
sns.lineplot(x='Step', y='Mutation Rate', data=df, ax=axs[0], color='blue')
axs[0].set_title('Trace Plot of Mutation Rates')
axs[0].set_ylabel('Mutation Rate')
axs[0].grid(True)

# Log-Likelihood Trace Plot
sns.lineplot(x='Step', y='Log Likelihood', data=df, ax=axs[1], color='green')
axs[1].set_title('Trace Plot of Log-Likelihoods')
axs[1].set_xlabel('MCMC Step')
axs[1].set_ylabel('Log Likelihood')
axs[1].grid(True)

plt.tight_layout()
plt.show()

# Step 3: Distribution Plots
plt.figure(figsize=(8, 6))
sns.histplot(df['Mutation Rate'], bins=30, kde=True, color='purple')
plt.title('Histogram and KDE of Mutation Rates')
plt.xlabel('Mutation Rate')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
sns.histplot(df['Log Likelihood'], bins=30, kde=True, color='orange')
plt.title('Histogram and KDE of Log-Likelihoods')
plt.xlabel('Log Likelihood')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Step 4: Autocorrelation Plots
plt.figure(figsize=(10, 4))
autocorrelation_plot(df['Mutation Rate'])
plt.title('Autocorrelation of Mutation Rates')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
autocorrelation_plot(df['Log Likelihood'])
plt.title('Autocorrelation of Log-Likelihoods')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.tight_layout()
plt.show()

# Step 5: Summary Statistics
print("Mutation Rate Summary Statistics:")
print(df['Mutation Rate'].describe())

print("\nLog-Likelihood Summary Statistics:")
print(df['Log Likelihood'].describe())



In [ ]:
# Verify that each tree has exactly one root
for i, tree in enumerate(ts.trees()):
    roots = [u for u in tree.nodes() if tree.parent(u) == tskit.NULL]
    if len(roots) != 1:
        print(f"Error: Tree {i} has {len(roots)} roots!")
    else:
        print(f"Tree {i} has a single root.")


In [ ]:
import numpy as np
import tskit
import logging

# Configure logging to see debug messages
logging.basicConfig(level=logging.INFO)

# Import your functions
from kernel import vfk0_imq_trees, make_imq_trees
from stein import kmat, ksd, create_integrand
from thinning import thin_trees

# Assuming you have run your MCMC sampler and have a Recorder object
# Extract data from the recorder

tree_sequence = recorder.tree_sequence()  # recorder.trees is a TreeSequence object
mutation_rates = recorder.mutation_rates  # List of mutation rates
log_likelihoods = recorder.log_likelihoods  # List of log-likelihoods
gradients = recorder.gradients  # List of gradients

# Extract individual trees from the tree sequence
trees = list(tree_sequence.trees())

# Now, ensure that the number of trees matches the number of mutation rates, log-likelihoods, and gradients
n_trees = len(trees)
assert n_trees == len(mutation_rates) == len(log_likelihoods) == len(gradients), "Mismatch in lengths"

# If your gradients are arrays, consider summarizing them if necessary
# For this example, we'll compute the norm of each gradient vector
gradients_norm = []
for grad in gradients:
    if isinstance(grad, np.ndarray):
        grad_norm = np.linalg.norm(grad)
    else:
        grad_norm = grad
    gradients_norm.append(grad_norm)

# Now we can proceed to test the functions

# Test the kernel function
index1 = 0
index2 = 1
tree1 = trees[index1]
tree2 = trees[index2]
grad1 = gradients_norm[index1]
grad2 = gradients_norm[index2]

# Set parameters for the kernel function
c = 1.0
beta = -0.5
lambda_ = 0.0  # Adjust as needed

kernel_value = vfk0_imq_trees(
    tree_x=tree1,
    tree_y=tree2,
    grad_x=grad1,
    grad_y=grad2,
    c=c,
    beta=beta,
    lambda_=lambda_
)
print(f"Kernel value between tree {index1} and tree {index2}: {kernel_value}")

# Create the kernel function using make_imq_trees
kernel_func = make_imq_trees(
    sample_trees=trees,
    gradients=gradients_norm,
    c=c,
    beta=beta,
    lambda_=lambda_
)

# Create the integrand function
integrand = create_integrand(
    trees=trees,
    gradients=gradients_norm,
    kernel_func=kernel_func
)

# Test the integrand function
kernel_value_single = integrand(0, 1)
print(f"Kernel value between trees 0 and 1: {kernel_value_single}")

# Compute the kernel matrix
n_trees_to_use = min(10, n_trees)  # Adjust as needed
kernel_matrix = kmat(integrand=integrand, n=n_trees_to_use)
print(f"Kernel matrix for the first {n_trees_to_use} trees:")
print(kernel_matrix)

# Perform thinning
n_select = 5  # Adjust as needed
selected_indices = thin_trees(
    trees=trees,
    gradients=gradients_norm,
    n_points=n_select,
    kernel_func=kernel_func
)
print(f"Selected indices after thinning: {selected_indices}")

# Examine the selected trees
for idx in selected_indices:
    tree = trees[idx]
    mutation_rate = mutation_rates[idx]
    log_likelihood = log_likelihoods[idx]
    gradient = gradients_norm[idx]
    print(f"Tree {idx}: Mutation Rate = {mutation_rate}, Log Likelihood = {log_likelihood}")
    # Visualize the tree using tskit
    print(tree.draw_text())


In [ ]:
ts.validate(ts)
